# Implementación Matriz de confusión y Kappa de Cohen
<p>Esta es una implementación para el tipo de dataset almacenado en data_1.csv o data_2.csv. La estructura es la siguiente:</p>
<ul>
<li><b>id_news:</b> ID de una noticia.</li>
<li><b>title:</b> Titulo de la noticia.</li>
<li><b>label:</b> El titulo es [POS, NEU, NEG]: positivo, neutral, negativo.</li>
<li><b>anotador:</b> Inicial del etiquetador y primer apellido.</li>
<li><b>label_ministerio:</b> Ministerio al que corresponde el titulo, abajo se ven los ministerios considerados.</li>
</ul>

<p>Para procesar los archivos de manera correcta los archivos deben tener las <b>columnas mencionadas</b>, con énfasis en <b>label</b> y <b>label_ministerio</b>, también deben tener las <b>mismas noticias etiquetadas</b>, así como un <b>anotador distinto</b>.</p>
<p>Se calculará y retornará la matriz de confusión y coheficiente de Cohen. <strong>Los cálculos son específicos para este tipo de dataset.</strong></p>

In [126]:
# read data.csv
import pandas as pd
import numpy as np

In [127]:
def Kohonen_(archivo_1, archivo_2, task):
    '''
    archivo_1.csv -> id_news, title, label, anotador, label_ministerio
    archivo_2.csv -> id_news, title, label, anotador, label_ministerio
    task -> 'label' or 'label_ministerio'

    Return:
    confusion_matrix -> np.array
    kappa -> float

    Se espera que anotador, label y label_ministerio sean distintos
    '''
    label = ['POS', 'NEU', 'NEG']
    label_ministerio = [
        'interior_y_seguridad_publica',
        'hacienda',
        'economia_fomento_turismo',
        'justicia_derechos_humanos',
        'salud',
        'mineria',
        'energia',
        'mujer_equidad_genero',
        'relaciones_exteriores',
        'secreteria_general_presidencia',
        'desarrollo_social_familia',
        'trabajo_prevision_social',
        'vivienda_urbanismo',
        'transportes_telecomunicaciones',
        'medio_ambiente',
        'culturas_artes_patrimonio',
        'defensa_nacional',
        'educacion',
        'obras_publicas',
        'agricultura',
        'bienes_nacionales',
        'deporte',
        'ciencia_tecnologia_conocimiento',
        'sin_etiqueta'
    ]


    # Intentamos leer los datos
    try:
        archivo_1 = pd.read_csv(archivo_1, sep=';')
        archivo_2 = pd.read_csv(archivo_2, sep=';')
    except:
        print('Error Archivo: No se encontró el archivo')
        return None
    print('OK: Archivos encontrados')
    
    # La variable 'task' corresponde a la columna que se quiere analizar?
    if (task != 'label' and task != 'label_ministerio'):
        print(f'Error Task: "{task}" no existe')
        return None

    print(f'OK: Etiqueta válida. Task: {task}')
    # Estamos analizando las mismas noticias?
    if (archivo_1['id_news'].values == archivo_2['id_news'].values).all():
        print('OK: Los archivos tienen las mismas noticias')
        
        # Mezclamos archivo_1 y archivo_2
        # Dejando: title, task, anotador_x
        archivo_1 = archivo_1[['title', task, 'anotador']]
        archivo_2 = archivo_2[['title', task, 'anotador']]
        archivo_1.rename(columns ={'title':'title',task:f'{task}_1','anotador':'anotador_1' }, inplace=True)
        archivo_2.rename(columns ={'title':'title',task:f'{task}_2','anotador':'anotador_2' }, inplace=True)
        
        # df: title	label_1	anotador_1	label_2	anotador_2
        # o bien df: title label_ministerio_1 anotador_1 label_ministerio_2 anotador_2
        df = archivo_1.merge(archivo_2, on='title')
        display(df.head())

        # --- En este punto ya tenemos label_1 y label_2 ---
        real = df[f'{task}_1'].values
        pred = df[f'{task}_2'].values

        if(task == 'label'):
            print(f'OK: Las etiquetas serán -> {label} -> {np.arange(len(label))}')

            # Consideraremos que el anotador 1 es el que tiene la etiqueta correcta
            # Por lo que si el anotador 1 y el anotador 2 tienen la misma etiqueta
            # entonces no hay discrepancia
            confusion_matrix = np.zeros((len(label), len(label)))
            for i in range(len(label)):
                for j in range(len(label)):
                    confusion_matrix[i, j] = np.sum((real == label[i]) & (pred == label[j]))
            print(f'\nExito (1/2): Retornando Matriz de Confusión')
            print(confusion_matrix)
            
            # Calculamos P0 de Kappa
            # P0 = Suma de aciertos (diagonal) dividido total
            suma_aciertos = np.sum(np.diag(confusion_matrix))
            total = np.sum(confusion_matrix)
            p0 = suma_aciertos / total

            # Calculamos Pe de Kappa
            # Pe = Suma de probabilidades
            # ej: pe_1= Sumar (fila i=0)/(total) * (Sumar columna i=0)/(total)
            # ej: pe_2= Sumar (fila i=1)/(total) * (Sumar columna i=1)/(total)
            # ej: pe_3= Sumar (fila i=1)/(total) * (Sumar columna i=1)/(total)
            # Valor Pe final: pe_1 + pe_2 + pe_3
            pe = 0
            for i in range(len(label)):
                pe += (np.sum(confusion_matrix[i, :]) / total) * (np.sum(confusion_matrix[:, i]) / total)

            # Calculamos Kappa
            kappa = (p0 - pe) / (1 - pe)
            print(f'\nExito (2/2): Retornando Kappa')
            print(kappa)
            return confusion_matrix, kappa


        if(task == 'label_ministerio'):
            print(f'OK: Las etiquetas serán -> ')
            for etiqueta in range(len(label_ministerio)):
                print(f'{etiqueta} -> {label_ministerio[etiqueta]}')

            confusion_matrix = np.zeros((len(label_ministerio), len(label_ministerio)))
            for i in range(len(label_ministerio)):
                for j in range(len(label_ministerio)):
                    confusion_matrix[i, j] = np.sum((real == label_ministerio[i]) & (pred == label_ministerio[j]))
            print(f'\nExito (1/2): Retornando Matriz de Confusión')
            print(confusion_matrix)
             
            # Calculamos P0 de Kappa
            # P0 = Suma de aciertos (diagonal) dividido total
            suma_aciertos = np.sum(np.diag(confusion_matrix))
            total = np.sum(confusion_matrix)
            p0 = suma_aciertos / total

            # Calculamos Pe de Kappa
            pe = 0
            for i in range(len(label_ministerio)):
                pe += (np.sum(confusion_matrix[i, :]) / total) * (np.sum(confusion_matrix[:, i]) / total)

            # Calculamos Kappa
            kappa = (p0 - pe) / (1 - pe)
            print(f'\nExito (2/2): Retornando Kappa')
            print(kappa)
            return confusion_matrix, kappa
        
        print('Error: Es posible que algo inesperado haya ocurrido.')
        return None
    else:
        print('Error Archivo: Los archivos tienen noticias diferentes')
        return None

In [128]:
nombre_archivo_1 = 'data_1.csv'
nombre_archivo_2 = 'data_2.csv'
confusion_matrix, kappa = Kohonen_(nombre_archivo_1, nombre_archivo_2, 'label')

OK: Archivos encontrados
OK: Etiqueta válida. Task: label
OK: Los archivos tienen las mismas noticias


,title,label_1,anotador_1,label_2,anotador_2
0,Diputados UDI piden información a municipalid...,NEU,agarnica,NEU,fruiz
1,Panquehue implementa aplicación lectora de pat...,POS,agarnica,NEU,fruiz
2,Dos personas fallecidas dejó accidente de trán...,NEG,agarnica,NEG,fruiz
3,Nuevo ataque incendiario dejó un camión destru...,NEG,agarnica,NEG,fruiz
4,PDi detuvo a 4 personas por tráfico en Laguna ...,POS,agarnica,POS,fruiz


OK: Las etiquetas serán -> ['POS', 'NEU', 'NEG'] -> [0 1 2]

Exito (1/2): Retornando Matriz de Confusión
[[14. 10.  0.]
 [ 2.  8.  0.]
 [ 0.  5. 10.]]

Exito (2/2): Retornando Kappa
0.4911423335369578


In [129]:
confusion_matrix, kappa = Kohonen_(nombre_archivo_1, nombre_archivo_2, 'label_ministerio')

OK: Archivos encontrados
OK: Etiqueta válida. Task: label_ministerio
OK: Los archivos tienen las mismas noticias


,title,label_ministerio_1,anotador_1,label_ministerio_2,anotador_2
0,Diputados UDI piden información a municipalid...,mujer_equidad_genero,agarnica,mujer_equidad_genero,fruiz
1,Panquehue implementa aplicación lectora de pat...,transportes_telecomunicaciones,agarnica,transportes_telecomunicaciones,fruiz
2,Dos personas fallecidas dejó accidente de trán...,transportes_telecomunicaciones,agarnica,transportes_telecomunicaciones,fruiz
3,Nuevo ataque incendiario dejó un camión destru...,interior_y_seguridad_publica,agarnica,interior_y_seguridad_publica,fruiz
4,PDi detuvo a 4 personas por tráfico en Laguna ...,interior_y_seguridad_publica,agarnica,interior_y_seguridad_publica,fruiz


OK: Las etiquetas serán -> 
0 -> interior_y_seguridad_publica
1 -> hacienda
2 -> economia_fomento_turismo
3 -> justicia_derechos_humanos
4 -> salud
5 -> mineria
6 -> energia
7 -> mujer_equidad_genero
8 -> relaciones_exteriores
9 -> secreteria_general_presidencia
10 -> desarrollo_social_familia
11 -> trabajo_prevision_social
12 -> vivienda_urbanismo
13 -> transportes_telecomunicaciones
14 -> medio_ambiente
15 -> culturas_artes_patrimonio
16 -> defensa_nacional
17 -> educacion
18 -> obras_publicas
19 -> agricultura
20 -> bienes_nacionales
21 -> deporte
22 -> ciencia_tecnologia_conocimiento
23 -> sin_etiqueta

Exito (1/2): Retornando Matriz de Confusión
[[11.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 